In [5]:
import numpy as np

# Neural Networks

In [81]:
class Neuralnets(object):
    
    def __init__(self,classes):
        self.classes = classes
        
    
    # Initialize the weights as a numpy array (All weights will be matrices)
    # Initialize the bias terms as a numpy array as well (Bias terms will be vectors)
    
    def buildNeuralNetworks(self,X,y,hL,nodes,iteration,eta):
        # SGD
        parameters = self.ParameterInitialization(X,y,nodes)
        weightStorage = parameters[0]
        biasStorage = parameters[1]
        for i in xrange(0,iteration):
            samples,featuress = X.shape
            for j in xrange(0,samples):
                x = X[i,]
            
                # Feed Forward
                # Store W,a,f'(z)
                #weightStorage = np.empty()
                valueStorage = np.empty()
                activeDerivStorage = np.empty()
                a = x
                for k in range(hL+1):
                    z =np.dot(np.transpose(W[k]),a) +b
                    a = self.ReLU(z)
                    valueStorage[k] = a
                    activeDerivStorage[a] = ReLUDeriv(z) 
                
                lossDeriv = self.crossEntropyLoss(y,a[L])
                outputError = np.dot(np.diag(self.softmaxDeriv(z)),np.transpose(lossDeriv))
                
                
                #Backpropagation 
                # Store delta at each iteration
                deltaStorage = np.empty()
                deltaStorage[hL] = outputError
                for l in range(1,hL):
                    layer = hL-l
                    delta = np.diag(fDerive[l])*W[l+1]*deltaStorage[layer+1]
                    deltaStorage[l] = delta
                 
                
                for k in range(1,hL+1):
                    weightGradient = np.dot(activeDerivStorage[k-1],np.transpose(deltaStorage[k]))
                    biasGradient = deltaStorage[k]
                    weightStorage[k-1] = weightStorage [k-1] - eta * weightGradient
                    biasStorage[k-1] = biasStorage [k-1] - eta * biasGradient
        return (weightStorage,biasStorage)
                    
                
                
    def ParameterInitialization(self,X,y,nodes):
        # Control the number of nodes in the weight initialization function
        # nodes variable should be a list of number of nodes in each hidden layer
        weightList = []
        biasList = []
        allNodes = [X.shape[1]]+nodes+[y.shape[1]]
        
        for i in range(1,len(allNodes)):
            weight = np.empty([allNodes[i-1],allNodes[i]])
            weight.fill(1/np.sqrt(allNodes[i-1]))
            weightList.append(weight)
            
            bias = np.zeros(allNodes[i])
            biasList.append(bias)
        return(weightList,biasList)
            
    
    
    def ReLU(self,z):
        return max(0,z)
    
    def softmax(self,Z):
        denom = sum(np.exp(Z))
        return(np.exp(Z)/denom)
    
    def ReLUDeriv(self,z):
        if z>0:
            Derive = 1
        else:
            # subgradient at zero was set to zero
            Derive = 0
        return(Derive)
    
    def softmaxDeriv(self,Z):
        n = len(Z)
        denom = sum(np.exp(Z))
        
        Jacobian = np.empty([n,n])
        for i in range(n):
            for j in range(i+1):
                if i == j:
                    Jacobian[i,j] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
                    Jacobian[j,i] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
                else:
                    Jacobian[i,j] = (np.exp(-(Z[i]+Z[j]))/denom)**2
                    Jacobian[j,i] = (np.exp(-(Z[i]+Z[j]))/denom)**2
        return (Jacobian)
                
    def crossEntropyLoss(self, y,guess):
        return(-sum(y/guess))
        
    

In [85]:
NNexercise = Neuralnets(2)
a = np.array([2.0,3.4,5.6,3.4,-0.1,0,-9])
NNexercise.softmaxDeriv(a)
NNexercise.ReLU(a.all)
b = np.array([[2.0,3.4],[5.6,3.4],[-0.1,0],[-9,0.1]])
f = np.vectorize(NNexercise.ReLU)  # or use a different name if you want to keep the original f

#result_array = f(b) 
#result_array
X = np.random.rand(4,2)
y = np.array([[0,1],[1,0],[1,0],[0,1]])
nodes = [10,2]
Params = NNexercise.ParameterInitialization(X,y,nodes)
Params[1]

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.]),
 array([ 0.,  0.])]

In [26]:
sum(np.exp(a))

n = len(a)
Z = a
denom = sum(np.exp(Z))
        
Jacobian = np.empty([n,n])
for i in range(n):
    for j in range(i+1):
        print i,j
        if i == j:
            print (np.exp(Z[i])/denom)
            print (np.exp(Z[i])/denom)**2
            Jacobian[i,j] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
            Jacobian[j,i] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
        else:
            print (np.exp(Z[i])/denom)
            print (np.exp(Z[i])/denom)**2
            Jacobian[i,j] = (np.exp(-(Z[i]+Z[j]))/denom)**2
            Jacobian[j,i] = (np.exp(-(Z[i]+Z[j]))/denom)**2


print Jacobian

0 0
0.0240076242008
0.000576366019766
1 0
0.0973557168631
0.00947813560592
1 1
0.0973557168631
0.00947813560592
2 0
0.878636658936
0.772002378426
2 1
0.878636658936
0.772002378426
2 2
0.878636658936
0.772002378426
[[  2.34312582e-02   2.15347600e-10   2.64389569e-12]
 [  2.15347600e-10   8.78775813e-02   1.60775462e-13]
 [  2.64389569e-12   1.60775462e-13   1.06634281e-01]]


# Convolutional Neural Networks

In [4]:
import tensorflow as tf

# Scratch

In [23]:
a = np.random.rand(3,3)
b = np.random.rand(3,2)
c = np.random.rand(4,3)
np.exp(c)

array([[ 1.46888017,  2.1098551 ,  1.32734838],
       [ 1.86130614,  1.91456954,  2.64912421],
       [ 2.47338498,  1.23116249,  1.17661799],
       [ 1.57488247,  2.20459446,  1.87048811]])

In [21]:
bigArray = [[a],[b],[c]]

In [22]:
bigArray[2]

[array([[ 0.66995296,  0.23646859,  0.53870775],
        [ 0.37648099,  0.56302431,  0.48723579],
        [ 0.47250215,  0.76336075,  0.20623173],
        [ 0.21332375,  0.99115811,  0.85635214]])]

In [3]:
for j in range(10,20):
    print 20-j

10
9
8
7
6
5
4
3
2
1


In [30]:
a = np.array([2,3.3,9,2.2])
b = np.array([2.9,7.3,1.9,3.2])
a/b

array([ 0.68965517,  0.45205479,  4.73684211,  0.6875    ])

In [7]:
np.exp(a)/sum(np.exp(a))

array([  9.07009851e-04,   3.32808822e-03,   9.94657078e-01,
         1.10782433e-03])

In [8]:
sum(np.exp(a)/sum(np.exp(a)))


1.0

In [44]:
W = np.empty()
for a in range(5):
    W[a] = np.rand(2,a)

TypeError: Required argument 'shape' (pos 1) not found

In [51]:
W = []
A = np.array([[2.9,7.3],[1.9,3.2]])
B = np.array([[2.9,7.3,23],[1.9,3.2,8]])
W.append(A)
W.append(B)

In [57]:
np.log(W[1])

array([[ 1.06471074,  1.98787435,  3.13549422],
       [ 0.64185389,  1.16315081,  2.07944154]])

In [58]:
np.array(1,[2,3])

TypeError: data type not understood

In [60]:
a = np.empty([2,3])
a.fill(1/np.sqrt(3))
a

array([[ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027]])

In [72]:
a = 2
s = [3]
u = [4]

b = s+[a]+u
b

[3, 2, 4]